# Comments

Here you can find a simple baseline model: https://www.kaggle.com/ragnar123/commonlit-readability-roberta-tf

This model has an out of folds root mean squared error of 0.5097

Here is the training script:

I tried to use tpu for training but the results are no good, hope this tensorflow baseline help to start this competition.

In [1]:
import re
import os
import numpy as np
import pandas as pd
import random
import math
import tensorflow as tf
import logging
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from tensorflow.keras import backend as K
from transformers import RobertaTokenizer, TFRobertaModel
from kaggle_datasets import KaggleDatasets
tf.get_logger().setLevel(logging.ERROR)
from kaggle_datasets import KaggleDatasets

In [2]:
# Configurations
# Number of folds for training
FOLDS = 5
selective = False
# Max length
MAX_LEN = 256#300#250
# Get the trained model we want to use
MODEL = '../input/tfroberta-base'
# Let's load our model tokenizer
tokenizer = RobertaTokenizer.from_pretrained(MODEL)

In [3]:
# This function tokenize the text according to a transformers model tokenizer
def regular_encode(texts, tokenizer, maxlen = MAX_LEN):
    enc_di = tokenizer.batch_encode_plus(
        texts,
        padding = 'max_length',
        truncation = True,
        max_length = maxlen,
    )
    
    return np.array(enc_di['input_ids'])

# This function encode our training sentences
def encode_texts(x_test, MAX_LEN):
    x_test = regular_encode(x_test.tolist(), tokenizer, maxlen = MAX_LEN)
    return x_test

# Function to build our model
def build_roberta_base_model(max_len = MAX_LEN):
    transformer = TFRobertaModel.from_pretrained(MODEL)
    input_word_ids = tf.keras.layers.Input(shape = (max_len, ), dtype = tf.int32, name = 'input_word_ids')
    sequence_output = transformer(input_word_ids)[0]
    # We only need the cls_token, resulting in a 2d array
    cls_token = sequence_output[:, 0, :]
    x = tf.keras.layers.Dense(256, activation = 'gelu', dtype = 'float32')(cls_token)
    output = tf.keras.layers.Dense(1, activation = 'linear', dtype = 'float32')(x)
    model = tf.keras.models.Model(inputs = [input_word_ids], outputs = [output])
    return model

# Function for inference
def roberta_base_inference(selective=False):
    # Read our test data
    df = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
    # Get text features
    x_test = df['excerpt']
    # Encode our text with Roberta tokenizer
    x_test = encode_texts(x_test, MAX_LEN)
    # Initiate an empty vector to store prediction
    predictions = np.zeros(len(df))
    # Predict with the 5 models (5 folds training)
    if selective:
        for i in FOLDS:
            print('\n')
            print('-'*50)
            print(f'Predicting with model {i}')
            # Build model
            model = build_roberta_base_model(max_len = MAX_LEN)
            # Load pretrained weights
            model.load_weights(f'../input/roberta-tf-adding-few-layers/Roberta_Base_123_{i}.h5')
            # Predict
            fold_predictions = model.predict(x_test).reshape(-1)
#             print(fold_predictions)
            # Add fold prediction to the global predictions
            predictions += fold_predictions / len(FOLDS)
    else:
        for i in range(FOLDS):
            print('\n')
            print('-'*50)
            print(f'Predicting with model {i + 1}')
            # Build model
            model = build_roberta_base_model(max_len = MAX_LEN)
            # Load pretrained weights
            model.load_weights(f'../input/roberta-tf-adding-few-layers/Roberta_Base_123_{i + 1}.h5')
            # Predict
            fold_predictions = model.predict(x_test).reshape(-1)
            # Add fold prediction to the global predictions
            predictions += fold_predictions / FOLDS
    # Save submissions
    df['target'] = predictions
    df[['id', 'target']].to_csv('submission.csv', index = False)
    return df

df = roberta_base_inference(selective)
df.head()



--------------------------------------------------
Predicting with model 1


All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at ../input/tfroberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.




--------------------------------------------------
Predicting with model 2


All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at ../input/tfroberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.




--------------------------------------------------
Predicting with model 3


...



--------------------------------------------------
Predicting with model 4


All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at ../input/tfroberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.




--------------------------------------------------
Predicting with model 5


All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at ../input/tfroberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


,id,url_legal,license,excerpt,target
0,c0f722661,NaN,NaN,My hope lay in Jack's promise that he would ke...,-0.233259
1,f0953f0a5,NaN,NaN,Dotty continued to go to Mrs. Gray's every nig...,-0.168042
2,0df072751,NaN,NaN,It was a bright and cheerful scene that greete...,-0.408615
3,04caf4e0c,https://en.wikipedia.org/wiki/Cell_division,CC BY-SA 3.0,Cell division is the process by which a parent...,-2.515218
4,0e63f8bea,https://en.wikipedia.org/wiki/Debugging,CC BY-SA 3.0,Debugging is the process of finding and resolv...,-1.812104
